# Project One: House Prices

Oliver James
DS 5033 Data Mining & Machine Learning

During the course of this project, you will learn to load, process, visualize and predict with housing data.
We will work through a series of steps:
1. Understand the Overarching Task
2. Acquire and Load the Data
3. Prepare the Data (for ML)
4. Selecting and Training an ML Model
5. Presenting an Analysis on the Solution

We will consolidate this notebook into three following steps:

* Broadly speaking, tasks 1-3 fall under Exploratory Data Analysis (EDA).
* Then, task four encompulates several steps for price price prediction (the modeling side of things).
* Finally, we have our analysis in task five.


Due to some difficulties in acquiring a dataset on Texas and San Antonio housing, we will be relying on an
older dataset. For the purposes of learning, the data is still relevant.

For this project, the data is on Californian houses (when they were still affordable).  
The csv is also attached to the assignment.

* The dataset is available from: https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html
* You can also get the csv from: https://github.com/ageron/data/blob/main/housing/housing.csv
* Read more here: https://www.kaggle.com/datasets/camnugent/california-housing-prices/data

In [ ]:
#install any libraries you may need

#This is a test round comment on branch test round. 

In [ ]:
#import all the relevant libraries here; include an inline comment for why you need them
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), plotting

### Exploratory Data Analysis (40%)

The task here is to construct a big picture view of the data set.

1. Download and load the dataset using pandas
2. Print statistics - use the head, info, describe functions
3. Visualize the data using the scatter_matrix function
4. Visualize the data according to geospatial data (HINT: plot using coordinate data)
5. Improve your geospatial plot: change parameters alpha, s, c, cmap. (your end result should have a colorbar with price and datapoints showing housing density with the hues indicating the price)

In [ ]:
#use multiple code blocks to organize your code

### Data Prep and Cleaning (20%)

Here, you will work to further investigate your features.

1. Check your features, are they in the correct units (median_income)
2. Find and fix missing values
3. Handle categorical data (HINT: OneHotEncoder)
4. Check the scaling for data. Does it need adjustment? (HINT: StandardScaler vs MinMaxScaler)
5. EXTRA CREDIT: how are you fixing (2)? Use SimpleImputer

In [ ]:
#use multiple code blocks to organize your code

### Price Prediction (20%)

Now you will create and test many ML models. The idea is to play with hyperparameters and model types.
You may find that some of your data prep needs further tweaking.

Now that you understand the big picture and have an intuition on how homes may be priced, you can move on to creating a price model.
Generally, you will want to follow the order of steps below (with 1&2 already completed):

1. Create a copy of your data.
2. On the copy, perform the data prep transformations.
3. Create a train & test split. Generally, 80-20 splits work pretty well.
4. Use a model you understand well. We recently learned how LinearRegression. How well does this model work?
5. In our introductory class, we also learned about the DecisionTreeRegressor. How does this perform?
6. EXTRA CREDIT: Model prices with the RandomForestRegressor model

The main idea here is to get used to programming and modeling on real data. Report your results and aim for
a ballpark Root Mean Squared Error less than 75K.

In [ ]:
#use multiple code blocks to organize your code

### Analysis (20%):

A critical component in science is communicating your results and explaining the reseasoning behind the results.
A good presentation here should include the following:  

1. An introduction to the dataset, any things we should know (e.g. how it was collected, common errors). 
2. What did you discover in your EDA? What do you do with missing values, outliers, etc.  
3. What kind of distribution is the data? Is there a skew or high concentration of houses in a certain range?  
4. What correlations were revealed in the analysis? e.g. footage and price, do they correlate positively?  
5. Feature selection: which feature worked for price predictions and what was noise? How did you determine?

The points above are for guidance; you can choose your template and structure.  
The idea is to present a short report (no word counts) that is structured. 
structured, clear, and concise.  
You can refer back to your figures and use external links to explain your insights.

### use this markdown cell to write a report

[here]

### Submission:

You need to prepare your ipynb/jupyter notebook for grading.
The two main tasks are ensuring all your cell outputs are present and that you convert the notebook to PDF.

The instructs will vary slightly based on the platform (collab, kaggle, anaconda, etc).
Generally, inside the notebook, you will want to:
1. Restart & clear all cell outputs (optional, may detect buggy program control flow)
2. Run all (must do; I need to see your code cell outputs!)

Next, you need to download the notebook as a PDF. Unfortunately, exporting as PDF is a bit tricky.
An easy work around:
1. Download the notebook. (all platforms allow the default .ipynb export)
2. https://onlineconvertfree.com/convert-format/ipynb-to-pdf/

If you are unable to upload as a PDF, submit the .ipynb. Do NOT upload a .py file.

### Rubric:

Please see the associated percentage allocations.  
In general, ensure your code runs correctly.  Make your the PDF upload includes your code ouputs.  
You will be given significant credit for documentation and pseudo-code.

For more details, please read the rubric PDF in the assignment files.